In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../src/')  
from scraper.session import login
from scraper.fetch import get_court_links
from scraper.parser import parse_cases


# getting court links doesnt need session as no log in... but it does want you to log in to view the deetts..
links = get_court_links()
# print(links)

# logging in
session = login()
print(f"cookies: {session.cookies}")

# parse cases using session data from login
parse_cases(session, links)


